In [1]:
from autocorrect import Speller
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer
import re
# nltk.download('punkt')
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

stopwords_set = set(stopwords.words('english'))
ps = PorterStemmer()

import time
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.proxy import *
from joblib import load, dump 
import pandas as pd # Pandas documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
import enchant
import string
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report

from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'

In [2]:
applications = [
    'facebook',
    'snapchat',
    'tiktok',
    'reddit',
    'spotify'
]

applications_to_urls = {
    applications[0] :'https://play.google.com/store/apps/details?id=com.facebook.katana&hl=en_US',
    applications[1]:'https://play.google.com/store/apps/details?id=com.snapchat.android&hl=en_US',
    applications[2]:'https://play.google.com/store/apps/details?id=com.zhiliaoapp.musically&hl=en_US',
    applications[3]:'https://play.google.com/store/apps/details?id=com.reddit.frontpage&hl=en_US',
    applications[4]:'https://play.google.com/store/apps/details?id=com.spotify.music'
}

urls_to_applications = {}
for (k, v) in applications_to_urls.items():
    urls_to_applications[v] = k
    
applications_to_save_files = {}
for (k,_) in applications_to_urls.items():
    applications_to_save_files[k] = k + '_reviews'

applications_to_reviews_by_sentence = {}
for (k,_) in applications_to_urls.items():
    applications_to_reviews_by_sentence[k] = applications_to_save_files[k] + '_by_sentence'

In [3]:
tiktok_reviews = pd.read_json('tiktok_reviews.json') 
facebook_reviews = pd.read_json('facebook_reviews.json') 
snapchat_reviews = pd.read_json('snapchat_reviews.json') 
reddit_reviews = pd.read_json('reddit_reviews.json')
spotify_reviews = pd.read_json('spotify_reviews.json')
review_json_files = ['tiktok_reviews.json', 'facebook_reviews.json', 'snapchat_reviews.json', 'reddit_reviews.json', 'spotify_reviews.json']

In [4]:
# json_review_df = pd.DataFrame(columns=['review_body', 'found_helpful', 'application'])
# for json_file in review_json_files:
#     application_review = pd.read_json(json_file)
#     application = json_file.split('_')[0]
#     for i in range(len(application_review)):
#         page_expansion = BeautifulSoup(application_review['review_body'][i], 'html.parser')
#         full_review = page_expansion.find('span', {'jsname':'fbQN7e'})
#         page_expansion = BeautifulSoup(application_review['found_helpful'][i], 'html.parser')
#         helpful_rating = page_expansion.find('div', class_='jUL89d')
#         helpful_rating = int(helpful_rating.text) if len(helpful_rating) != None else 0
#         if helpful_rating > 200:
#             json_review_df = json_review_df.append({'review_body': full_review.text, 'found_helpful': helpful_rating, 'application': application}, ignore_index=True)
# print(len(json_review_df))

In [5]:
# # split by sentence
# json_reviews_by_sent = pd.DataFrame(columns=['review_num', 'review_body', 'application', 'classification'])
# for i in range(len(json_review_df)):
#     review = json_review_df['review_body'][i]
#     sentences = nltk.tokenize.sent_tokenize(review)
#     for sentence in sentences:
#         json_reviews_by_sent = json_reviews_by_sent.append({'review_num': i, 'review_body': sentence, 'application': json_review_df['application'][i],'classification': None},
#                                                           ignore_index=True) 
# print(len(json_reviews_by_sent))
# json_reviews_by_sent = json_reviews_by_sent.sample(frac=1).reset_index(drop=True)

In [6]:
# # label reviews as informative or uninformative
# # DO NOT RUN THIS IF DATA IS LABELED ALREADY
# for i in range(1000):
#     # print("======================")
#     if json_reviews_by_sent['classification'][i] == None: 
#         print('Review ' + str(i))
#         print(json_reviews_by_sent['review_body'][i])
#         classification = input('Informative or non-informative? ') 
#         if classification == 'n' or classification == 'i':
#             classification = 'non-informative' if classification == 'n' else 'informative'
#         if classification == 'v':
#             classification = 'vague'
#         json_reviews_by_sent['classification'][i] = classification
#     dump(json_reviews_by_sent, 'json_review_dataframe_by_sent', compress=True)

In [7]:
def clean_review(sentence):
    no_punc = [c for c in sentence if c not in string.punctuation]
    no_punc = ''.join(no_punc)
    no_stopwords = [w.lower() for w in no_punc.split() if (w not in stopwords_set) and (len(re.search('^\s*[0-9]*', w)[0]) == 0)]    
    stemmed_words = [ps.stem(w) for w in no_stopwords]
    return stemmed_words

def clean_review_len(sentence):
    return len(clean_review(sentence))

In [10]:
analyzer = SentimentIntensityAnalyzer()
i = 100

def get_sentiment(s):
    vs = analyzer.polarity_scores(s)
    if vs['compound'] >= 0.05:
        return 2
    elif vs['compound'] <= -0.05:
        return 0
    else:
        return 1
def get_strict_sentiment(s):
    vs = analyzer.polarity_scores(s)
    if vs['compound'] >= 0.0:
        return 2
    else:
        return 0


In [14]:
json_reviews_by_sent = load('json_review_dataframe_by_sent')
json_reviews_by_sent['sentiment'] = json_reviews_by_sent['review_body'].apply(get_sentiment)
json_word_freq_table = create_word_freq_table(json_reviews_by_sent)
json_word_freq_table['_sentiment'] = json_reviews_by_sent['sentiment']
json_word_freq_table['_classification'] = json_reviews_by_sent['classification']

In [15]:
# json_reviews_by_sent['clean_review_body'] = json_reviews_by_sent['review_body'].apply(clean_review)
# json_reviews_by_sent['review_body'] = json_reviews_by_sent['review_body'].apply(fix_tiktok)
# json_reviews_by_sent = load('json_review_dataframe')
print('Num informative: ', len(json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'informative')]))
print('Num non-informative: ', len(json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'non-informative')]))
print('Num vague: ', len(json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'vague')]))
# perform whatever manipulations you want on the json_reviews_by_sent dataframe
non_informative_df = json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'non-informative')]
informative_df = json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'informative')]
classification_df = informative_df.append(non_informative_df[:361], ignore_index=True)

# drop irrelevant columns
classification_df.drop(['review_num', 'application', 'length'], axis=1, inplace=True)


# With words_freq_table
non_informative_df = json_word_freq_table[(json_word_freq_table['_classification'] == 'non-informative')]
informative_df = json_word_freq_table[(json_word_freq_table['_classification'] == 'informative')]
classification_df = informative_df.append(non_informative_df[:361], ignore_index=True)
# classification_df = classification_df.append(json_word_freq_table[(json_word_freq_table['_classification'] == 'vague')], ignore_index=True)

# trying 3 categories
# classification_df = json_word_freq_table[:1000]

Num informative:  361
Num non-informative:  442
Num vague:  131


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from mlxtend.preprocessing import DenseTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# msg_train, msg_test, label_train, label_test = train_test_split(classification_df.drop('classification', axis=1), classification_df['classification'], test_size=0.3, random_state=42)
msg_train, msg_test, label_train, label_test = train_test_split(classification_df.drop('_classification', axis=1), classification_df['_classification'], test_size=0.3, random_state=42)
param_grid = {'classifier__C': [0.1,1, 10, 100, 1000], 'classifier__gamma': [1,0.1,0.01,0.001,0.0001], 'classifier__kernel': ['rbf']} 
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 

In [17]:
# from sklearn.pipeline import Pipeline

# pipeline1 = Pipeline([
#     ('bow', CountVectorizer(analyzer=clean_review)),
#     ('tfidf', TfidfTransformer()),
#     ('classifier', MultinomialNB())
# ])

# pipeline2 = Pipeline([
#     ('bow', CountVectorizer(analyzer=clean_review)),
#     ('tfidf', TfidfTransformer()),
#     ('classifier', BernoulliNB())
# ])


# pipeline3 = Pipeline([
#     ('bow', CountVectorizer(analyzer=clean_review)),
#     ('tfidf', TfidfTransformer()),
#     ('classifier', SVC(C=10, gamma=1, kernel='rbf'))
# ])

# # grid = GridSearchCV(pipeline3, param_grid, refit=True, verbose=0, cv=10)
# # 0.7557142857142857
# # {'classifier__C': 10, 'classifier__gamma': 1, 'classifier__kernel': 'rbf'}

# pipeline4 = Pipeline([
#     ('bow', CountVectorizer(analyzer=clean_review)),
#     ('tfidf', TfidfTransformer()),
#     ('classifier', RandomForestClassifier(n_estimators=200))
# ])

############################################################################################################################################

mnb = MultinomialNB()
mnb.fit(msg_train, label_train)
predictions1 = mnb.predict(msg_test)
print('Multinomial')
print(classification_report(predictions1, label_test))

rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(msg_train, label_train)
predictions2 = rfc.predict(msg_test)
print('Random Forest Classifier')
print(classification_report(predictions2, label_test))

# grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=0, cv=10)
# grid.fit(msg_train, label_train)
# predictions3 = grid.predict(msg_test)
# print('Support Vector Machine')
# print(grid.best_params_)
# print(grid.best_estimator_)
# print(classification_report(predictions3, label_test))


svm = SVC(C = 10, gamma= 0.01)
svm.fit(msg_train, label_train)
predictions3 = svm.predict(msg_test)
print('SVM')
print(classification_report(predictions3, label_test))

bnb = BernoulliNB()
bnb.fit(msg_train, label_train)
predictions4 = bnb.predict(msg_test)
print('Bernoulli')
print(classification_report(predictions4, label_test))

Multinomial
                 precision    recall  f1-score   support

    informative       0.86      0.76      0.81       129
non-informative       0.70      0.82      0.75        88

       accuracy                           0.78       217
      macro avg       0.78      0.79      0.78       217
   weighted avg       0.79      0.78      0.79       217

Random Forest Classifier
                 precision    recall  f1-score   support

    informative       0.70      0.93      0.80        86
non-informative       0.94      0.74      0.83       131

       accuracy                           0.82       217
      macro avg       0.82      0.84      0.81       217
   weighted avg       0.85      0.82      0.82       217

SVM
                 precision    recall  f1-score   support

    informative       0.79      0.94      0.86        96
non-informative       0.94      0.80      0.87       121

       accuracy                           0.86       217
      macro avg       0.87      0.87   

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
# pipeline1.fit(msg_train, label_train)
# pipeline2.fit(msg_train, label_train)
# pipeline3.fit(msg_train, label_train)
# pipeline4.fit(msg_train, label_train)

In [20]:
scores1 = cross_val_score(mnb, msg_train, label_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores1.mean(), scores1.std() * 2))
scores2 = cross_val_score(rfc, msg_train, label_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores2.mean(), scores2.std() * 2))
scores3 = cross_val_score(svm, msg_train, label_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores3.mean(), scores3.std() * 2))
scores4 = cross_val_score(bnb, msg_train, label_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores4.mean(), scores4.std() * 2))

Accuracy: 0.76 (+/- 0.11)
Accuracy: 0.79 (+/- 0.14)
Accuracy: 0.83 (+/- 0.12)
Accuracy: 0.79 (+/- 0.14)


In [21]:
predictions1 = mnb.predict(msg_test)
predictions2 = rfc.predict(msg_test)
predictions3 = svm.predict(msg_test)
predictions4 = bnb.predict(msg_test)

In [22]:
# json_reviews_by_sent['length'] = json_reviews_by_sent['review_body'].apply(clean_review_len)

In [23]:
# json_reviews_by_sent.hist(column='length', by='classification', bins=50,figsize=(12,4))
# json_reviews_by_sent.head()

In [24]:
from sklearn.metrics import classification_report
print('Multinomial')
print(classification_report(predictions1, label_test))
print('RFC')
print(classification_report(predictions2, label_test))
print('SVM')
print(classification_report(predictions3, label_test))
print('Bernoulli')
print(classification_report(predictions4, label_test))


Multinomial
                 precision    recall  f1-score   support

    informative       0.86      0.76      0.81       129
non-informative       0.70      0.82      0.75        88

       accuracy                           0.78       217
      macro avg       0.78      0.79      0.78       217
   weighted avg       0.79      0.78      0.79       217

RFC
                 precision    recall  f1-score   support

    informative       0.70      0.93      0.80        86
non-informative       0.94      0.74      0.83       131

       accuracy                           0.82       217
      macro avg       0.82      0.84      0.81       217
   weighted avg       0.85      0.82      0.82       217

SVM
                 precision    recall  f1-score   support

    informative       0.79      0.94      0.86        96
non-informative       0.94      0.80      0.87       121

       accuracy                           0.86       217
      macro avg       0.87      0.87      0.86       217
   

In [34]:
pred = svm.predict(json_word_freq_table[1001:].drop('_classification', axis=1).fillna(0))
for i in range(len(pred)):
    if json_reviews_by_sent['classification'][1001+i] == None:
        json_reviews_by_sent['classification'][1001+i] = pred[i]
dump(json_reviews_by_sent, 'json_review_dataframe_by_sent', compress=True)

['json_review_dataframe_by_sent']

In [36]:
dump(svm, 'best_model', compress=True)

['best_model']

In [26]:
# i_df = json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'informative')]
# ni_df = json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'non-informative')]
# v_df = json_reviews_by_sent[(json_reviews_by_sent['classification'] == 'vague')]
# print(i_df['review_body'][1])
# print(v_df['review_body'][50])
# print(ni_df['review_body'][4])


# Topic Modeling

In [27]:
def create_word_freq_table(input_df):
    bow_transformer = CountVectorizer(analyzer=clean_review).fit(input_df['review_body'])
    transformed_input = bow_transformer.transform(input_df['review_body'])
    count_vect_df = pd.DataFrame(transformed_input.todense(), columns=bow_transformer.get_feature_names())
    return count_vect_df

In [28]:
topic_modeling_df = json_reviews_by_sent[json_reviews_by_sent['classification'] == 'informative']
json_bow = create_word_freq_table(topic_modeling_df)
json_bow
tdm = json_bow.transpose()

In [29]:
i = 0
shuffled_df = topic_modeling_df.sample(frac=1).reset_index(drop=True)
while i < 100:
    try:
        i += 1
        print(shuffled_df['review_body'][i])
    except KeyError:
        pass

Sometimes the radius selection tool works, sometimes it shows a 200km radius even though you're trying to select 10km.
And are stuck with it, but we know the interface is laughable right?
Like of you only put 6 songs on a playlist, it adds suggested songs and I hate that so much.
Can't upload video over 1080p.
When you click them; it takes you to the right place but wont let you see what the notification was for.
I normally have no problems at all with Facebook an I would have given 5 stars but over the last 2 wks it's been a complete nightmare..home page is constantly stuck on the same feed notifications are no longer working ,see 1st is the same I've not had one at the beginning of the home screen ...I've try'd contacting you on several occasions to see I'd theres any way it can be sorted lol I'm not even getting a reply from that ..please please please for my sanity help
Can't push several different buttons at time: admin button off and on, clicking on profiles, saving and glitches,

In [30]:
from gensim import matutils, models
import scipy.sparse

In [31]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [32]:
bow_transformer = CountVectorizer(analyzer=clean_review).fit(topic_modeling_df['review_body'])
id2word = dict((v, k) for k, v in bow_transformer.vocabulary_.items())

In [47]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=7, passes=20)
lda.print_topics()

Help on LdaModel in module gensim.models.ldamodel object:

class LdaModel(gensim.interfaces.TransformationABC, gensim.models.basemodel.BaseTopicModel)
 |  LdaModel(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<class 'numpy.float32'>)
 |  
 |  Train and use Online Latent Dirichlet Allocation (OLDA) models as presented in
 |  `Hoffman et al. :"Online Learning for Latent Dirichlet Allocation" <https://www.di.ens.fr/~fbach/mdhnips2010.pdf>`_.
 |  
 |  Examples
 |  -------
 |  Initialize a model using a Gensim corpus
 |  
 |  .. sourcecode:: pycon
 |  
 |      >>> from gensim.test.utils import common_corpus
 |      >>>
 |      >>> lda = LdaModel(common_corpus, num_topics=10)
 |  
 |  You can then infer t

In [48]:
help(lda)

Help on LdaModel in module gensim.models.ldamodel object:

class LdaModel(gensim.interfaces.TransformationABC, gensim.models.basemodel.BaseTopicModel)
 |  LdaModel(corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01, random_state=None, ns_conf=None, minimum_phi_value=0.01, per_word_topics=False, callbacks=None, dtype=<class 'numpy.float32'>)
 |  
 |  Train and use Online Latent Dirichlet Allocation (OLDA) models as presented in
 |  `Hoffman et al. :"Online Learning for Latent Dirichlet Allocation" <https://www.di.ens.fr/~fbach/mdhnips2010.pdf>`_.
 |  
 |  Examples
 |  -------
 |  Initialize a model using a Gensim corpus
 |  
 |  .. sourcecode:: pycon
 |  
 |      >>> from gensim.test.utils import common_corpus
 |      >>>
 |      >>> lda = LdaModel(common_corpus, num_topics=10)
 |  
 |  You can then infer t

In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
pyLDAvis.genesim.prepare(lda, corpus, dictionary)
pyLDAvis.display(lda_display10)

In [244]:
from nltk import word_tokenize, pos_tag

In [245]:
def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [246]:
informative_clean = pd.DataFrame(columns=['review'])
for i in range(len(json_reviews_by_sent)):
    if json_reviews_by_sent['classification'][i] == 'informative':
        clean_sentence = []
        for w in json_reviews_by_sent['review_body'][i].split():
            if w not in stopwords_set:
                clean_sentence.append(w)
        s = ' '.join(clean_sentence)
        informative_clean = informative_clean.append({'review': s}, ignore_index=True)        

In [247]:
data_nouns = pd.DataFrame(informative_clean['review'].apply(nouns))

In [248]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/qh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [249]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns['review'])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,a10s,a50,abit,access,account,accounts,acount,action,activation,activites,...,yadda,yall,year,years,yell,yesterday,yolo,youtube,zoom,zooms
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [250]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [251]:

# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.075*"notifications" + 0.041*"app" + 0.017*"list" + 0.015*"phone" + 0.015*"feed" + 0.013*"ads" + 0.012*"snaps" + 0.012*"version" + 0.012*"work" + 0.011*"button"'),
 (1,
  '0.030*"friends" + 0.021*"screen" + 0.019*"page" + 0.018*"posts" + 0.018*"videos" + 0.018*"post" + 0.015*"songs" + 0.013*"photo" + 0.013*"story" + 0.013*"play"'),
 (2,
  '0.051*"app" + 0.035*"notification" + 0.015*"music" + 0.015*"camera" + 0.013*"reason" + 0.013*"message" + 0.012*"way" + 0.012*"pages" + 0.012*"posts" + 0.011*"users"'),
 (3,
  '0.027*"video" + 0.022*"facebook" + 0.020*"problem" + 0.019*"app" + 0.019*"phone" + 0.018*"times" + 0.015*"option" + 0.013*"posts" + 0.013*"messages" + 0.013*"account"')]

In [252]:
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [253]:
data_nouns_adj = pd.DataFrame(informative_clean['review'].apply(nouns_adj))
data_nouns_adj

,review
0,new notifications settings
1,okay overall better change thumbnail items cur...
2,app fine voice constant ads
3,today latest pictures month November
4,Overall app nice someone log account phone Oop...
5,months
6,order fix reinstalled app
7,different YouTube ads right video NEVER EVER i...
8,able view notifications
9,way unlock phone bring app starts beginning song


In [254]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj['review'])
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index

# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [274]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=200)
ldana.print_topics()

[(0,
  '0.021*"notification" + 0.016*"app" + 0.016*"snap" + 0.016*"messages" + 0.015*"message" + 0.015*"notifications" + 0.015*"screen" + 0.014*"videos" + 0.014*"snaps" + 0.013*"post"'),
 (1,
  '0.046*"app" + 0.016*"posts" + 0.014*"facebook" + 0.013*"way" + 0.011*"pages" + 0.011*"thing" + 0.010*"good" + 0.010*"video" + 0.008*"post" + 0.008*"videos"'),
 (2,
  '0.032*"app" + 0.027*"update" + 0.024*"posts" + 0.018*"facebook" + 0.013*"camera" + 0.013*"feed" + 0.013*"friends" + 0.012*"days" + 0.011*"content" + 0.011*"version"'),
 (3,
  '0.039*"notifications" + 0.023*"app" + 0.020*"video" + 0.019*"account" + 0.016*"problem" + 0.015*"new" + 0.015*"phone" + 0.014*"list" + 0.013*"friends" + 0.012*"comments"')]

In [263]:
from rasa_nlu.training_data  import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [264]:
train_data = load_data('rasa_dataset.json')

In [265]:
trainer = Trainer(config.load('config_spacy.yaml'))

In [266]:
trainer.train(train_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

In [267]:
model_directory = trainer.persist('./projects/')

In [268]:
import spacy
nlp = spacy.load('en')

In [269]:
from rasa_nlu.model import Metadata, Interpreter
interpreter = Interpreter.load(model_directory)

In [272]:
for i in range(len(informative_clean)):
    try:
        print(interpreter.parse(text=informative_df['review'][i]))
    except KeyError:
        pass

{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence': 0.0}, 'entities': [], 'text': ''}
{'intent': {'name': None, 'confidence'

In [340]:
import sys
print(sys.path)

['/Users/qh/Documents/capstone', '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/geos', '/usr/local/Cellar/numpy/1.16.0/libexec/nose/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/IPython/extensions', '/Users/qh/.ipython']


In [45]:
informative_df = json_reviews_by_sent[json_reviews_by_sent['classification'] == 'informative']
for i in range(400, 500):
    try:
        print(informative_df['review_body'][i])
    except KeyError:
        pass

When I start to scroll down it starts showing me videos I've never seen before!
Now, after deleting, I'm trying to sign back in to the app, and even that's not working.
Though navigation to other things is weird, and when you upvote or downvote when the picture is opened and then close picture it then it's not upvoted anymore, but still a great community.
Also, the cover art thumbnails are way too small.
Using the internet browser is a better experience than this app.
Sometimes duplicated notifications.
I would really love it if you could zoom in on stickers more.
When I'm reading comments on a video, itll sometimes continue to play the audio on the video which is rather annoying.
When i finished with the account identifying, it shows a blank page fill with languages.
Facebook has so many problems%uD83D%uDE1E Pictures & videos r not uoload clearly as phone..its bad..it should be clear hd colorful.. And fb should make sure every pictures security not just locked profile pro pics..each n

In [39]:

for i in range(100):


I really like this app but alot of things could be fixed.


KeyError: -1